In [1]:
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Exploring data from 'bsas.tif'

with rasterio.open('bsas.tif') as src:
    data = src.read(1, masked=True)

plt.figure(figsize=(10,8))
plt.imshow(data, cmap= 'summer')
plt.colorbar(label='Elevación (m)')
plt.title("Modelo de Elevación digital")
plt.show()